In [ ]:
import numpy as np
import torch
import torch.nn as nn

### Kernel

In [ ]:
import torch.nn.functional as F
def affinity_from_flow(flows, directions_array, flow_strength = 1, sigma=1):
  """Compute probabilities of transition in the given directions based on the flow. 

  Parameters
  ----------
  flows : torch tensor of shape n_points x n_dims
      _description_
  directions_array : torch tensor of shape n_directions x n_points x n_dims. Assumed to be normalized.
      _description_
  sigma : int, optional
      kernel bandwidth, by default 1
  returns (n_points)
  """
  assert len(flows.shape) == 2 # flows should only have one dimension
  assert len(directions_array.shape) > 1 and len(directions_array.shape) < 4
  n_directions = directions_array.shape[0]
  # Normalize directions
  length_of_directions = torch.linalg.norm(directions_array,dim=-1)
  normed_directions = F.normalize(directions_array,dim=-1)
  # and normalize flows # TODO: Perhaps reconsider
  flows = F.normalize(flows,dim=-1)

  if len(directions_array) == 1: # convert to 2d array if necessary
    directions_array = directions_array[:,None] 
  # compute dot products as matrix multiplication
  dot_products = (normed_directions * flows).sum(-1)
  # take distance between flow projected onto direction and the direction
  distance_from_flow = (torch.linalg.norm(flows,dim=1)**2).repeat(n_directions,1) - dot_products
  # take absolute value
  distance_from_flow = torch.abs(distance_from_flow)
  # print('shape of dff',distance_from_flow.shape)
  # add to this the length of each direction
  distance_from_flow = flow_strength*distance_from_flow + length_of_directions
  # put the points on rows, directions in columns
  distance_from_flow = distance_from_flow.T
  # take kernel of distances
  kernel =  torch.exp(-distance_from_flow/sigma)
  # normalize kernel
  # kernel /= torch.sum(kernel,axis=1)
  return kernel

In [ ]:
'''
import torch.nn.functional as F
def affinity_from_flow(flows, directions_array, flow_strength = 1, sigma=1):
  """Compute probabilities of transition in the given directions based on the flow. 

  Parameters
  ----------
  flows : torch tensor of shape n_points x n_dims
      _description_
  directions_array : torch tensor of shape n_directions x n_points x n_dims. Assumed to be normalized.
      _description_
  sigma : int, optional
      kernel bandwidth, by default 1
  returns (n_points)
  """
  assert len(flows.shape) == 2 # flows should only have one dimension
  assert len(directions_array.shape) > 1 and len(directions_array.shape) < 4
  n_directions = directions_array.shape[0]
  # Normalize directions
  length_of_directions = torch.linalg.norm(directions_array,dim=-1)
  normed_directions = F.normalize(directions_array,dim=-1)
  # and normalize flows # TODO: Perhaps reconsider
  flows = F.normalize(flows,dim=-1)

  if len(directions_array) == 1: # convert to 2d array if necessary
    directions_array = directions_array[:,None] 
  # compute dot products as matrix multiplication
  dot_products = (normed_directions * flows).sum(-1)
  # take distance between flow projected onto direction and the direction
  distance_from_flow = (torch.linalg.norm(flows,dim=1)**2).repeat(n_directions,1) - dot_products
  # take absolute value
  distance_from_flow = torch.abs(distance_from_flow)
  # print('shape of dff',distance_from_flow.shape)
  # add to this the length of each direction
  distance_from_flow = flow_strength*distance_from_flow + length_of_directions
  # put the points on rows, directions in columns
  distance_from_flow = distance_from_flow.T
  # take kernel of distances
  kernel =  torch.exp(-distance_from_flow/sigma)
  # normalize kernel
  # kernel /= torch.sum(kernel,axis=1)
  return kernel
'''

'\nimport torch.nn.functional as F\ndef affinity_from_flow(flows, directions_array, flow_strength = 1, sigma=1):\n  """Compute probabilities of transition in the given directions based on the flow. \n\n  Parameters\n  ----------\n  flows : torch tensor of shape n_points x n_dims\n      _description_\n  directions_array : torch tensor of shape n_directions x n_points x n_dims. Assumed to be normalized.\n      _description_\n  sigma : int, optional\n      kernel bandwidth, by default 1\n  returns (n_points)\n  """\n  assert len(flows.shape) == 2 # flows should only have one dimension\n  assert len(directions_array.shape) > 1 and len(directions_array.shape) < 4\n  n_directions = directions_array.shape[0]\n  # Normalize directions\n  length_of_directions = torch.linalg.norm(directions_array,dim=-1)\n  normed_directions = F.normalize(directions_array,dim=-1)\n  # and normalize flows # TODO: Perhaps reconsider\n  flows = F.normalize(flows,dim=-1)\n\n  if len(directions_array) == 1: # convert

### Calculate Directions between point i to point j and apply Kernel

In [ ]:
def affinity_matrix_from_pointset_to_pointset(pointset1, pointset2, flows,n_neighbors=None,sigma=0.5, flow_strength=1):
  """Compute affinity matrix between the points of pointset1 and pointset2, using the provided flow.

  Parameters
  ----------
  pointset1 : torch tensor, n1 x d
      The first pointset, to calculate affinities *from*
  pointset2 : torch tensor, n2 x d
      The second pointset, to calculate affinities *to* (from pointset1)
  flow : a function that, when called at a point, gives the flow at that point
  n_neighbors : number of neighbors to include in affinity computations. All neighbors beyond it are given affinity zero
  (currently not implemented)

  Returns:
  Affinity matrix: torch tensor of shape n1 x n2
  """
  # Calculate the directions from point i in pointset 1 to point j in pointset 2
  n1 = pointset1.shape[0]
  n2 = pointset2.shape[0]
  P2 = pointset2[:,:,None].repeat(1,1,n1)
  P1 = pointset1.T.repeat(n2,1,1)
  P3 = (P2 - P1)
  P3 = P3.transpose(1,2)
  # compute affinities from flows and directions
  affinities = affinity_from_flow(flows,P3,sigma=sigma,flow_strength=flow_strength)
  return affinities


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch_geometric
from tqdm import trange
from directed_graphs.utils import diffusion_matrix_from_graph
import numpy as np
import matplotlib.pyplot as plt

class DiffusionFlowEmbedder(torch.nn.Module):
	def __init__(self, X, flows, t = 4, sigma_graph = 0.5, sigma_embedding=0.5, embedding_dimension=2, device=torch.device('cpu'), autoencoder_shape = [100,10], flow_artist_shape = [10,20,10], flow_strength=1):
		"""Flow Embedding with diffusion

		Parameters
		----------
		X : torch tensor n_points x n_dim
			data matrix
		flows : torch tensor n_points x n_dim
			The flow at each point
		t : int
			Loss is computed with the diffusion operator powered to this number
		sigma in [0,1]
			Kernel bandwidth in the embedding
		"""
		# initialize parameters
		super(DiffusionFlowEmbedder, self).__init__()
		self.X = X
		self.ground_truth_flows = flows
		self.t = t
		self.sigma_embedding = sigma_embedding
		self.sigma_graph = sigma_graph
		self.nnodes = X.shape[0]
		self.data_dimension = X.shape[1]
		self.losses = []
		self.eps = 0.001
		if flow_strength == "learnable":
			self.flow_strength = nn.Parameter(torch.tensor(1.0))
		else:
			self.flow_strength = flow_strength	

		self.embedding_dimension = embedding_dimension
		# set device (used for shuffling points around during visualization)
		self.device = device
		# Compute P^t of the graph, the powered diffusion matrix
		# TODO: This can be optimized using landmarks, etc. For now it's straight sparse matrix multiplication
		# TODO: Migrate to a specialized function for dataset affinity calculation, with automatic kernel bandwidth selection, and the like
		self.P_graph = affinity_matrix_from_pointset_to_pointset(X,X,flows,sigma=sigma_graph)
		self.P_graph_t = torch.matrix_power(self.P_graph,self.t)
		# Flow field
		self.FlowArtist = nn.Sequential(nn.Linear(self.embedding_dimension, flow_artist_shape[0]),
		                       nn.Tanh(),
		                       nn.Linear(flow_artist_shape[0], flow_artist_shape[1]),
		                       nn.Tanh(),
													 nn.Linear(flow_artist_shape[1], flow_artist_shape[2]),
													 nn.Tanh(),
		                       nn.Linear(flow_artist_shape[2], self.embedding_dimension))
		# Autoencoder to embed the points into a low dimension
		self.encoder = nn.Sequential(nn.Linear(self.data_dimension, autoencoder_shape[0]),
															nn.ReLU(),
															nn.Linear(autoencoder_shape[0], autoencoder_shape[1]),
															nn.ReLU(),
															nn.Linear(autoencoder_shape[1], self.embedding_dimension))
		self.decoder = nn.Sequential(nn.Linear(self.embedding_dimension, autoencoder_shape[1]),
															nn.ReLU(),
															nn.Linear(autoencoder_shape[1], autoencoder_shape[0]),
															nn.ReLU(),
															nn.Linear(autoencoder_shape[0], self.data_dimension))
		# training ops
		self.KLD = nn.KLDivLoss(reduction='batchmean',log_target=False)
		self.MSE = nn.MSELoss()
		self.optim = torch.optim.Adam(self.parameters())
									

	def compute_embedding_P(self):
		A = affinity_matrix_from_pointset_to_pointset(self.embedded_points,self.embedded_points,flows = self.FlowArtist(self.embedded_points), sigma = self.sigma_embedding, flow_strength=self.flow_strength)
		# print("affinities ",A)
		# flow
		self.P_embedding = torch.diag(1/A.sum(axis=1)) @ A
		# power it
		self.P_embedding_t = torch.matrix_power(self.P_embedding,self.t)

	def loss(self):
		self.embedded_points = self.encoder(self.X)
		# print(self.embedded_points)
		# compute embedding diffusion matrix
		self.compute_embedding_P()
		# compute autoencoder loss
		X_reconstructed = self.decoder(self.embedded_points)
		reconstruction_loss = self.MSE(X_reconstructed, self.X)
		# print("recon loss",reconstruction_loss)
		# take KL divergence between it and actual P
		# print("embedding p",self.P_embedding_t)
		log_P_embedding_t = torch.log(self.P_embedding_t)
		# print(log_P_embedding_t)
		if log_P_embedding_t.is_sparse:
			diffusion_loss = self.KLD(log_P_embedding_t.to_dense(),self.P_graph_t.to_dense())
		else:
			diffusion_loss = self.KLD(log_P_embedding_t,self.P_graph_t)
		# print("diffusion loss is",diffusion_loss)
		cost = diffusion_loss + reconstruction_loss
		# print(f"cost is KLD {diffusion_loss} with recon {reconstruction_loss}")
		self.losses.append([diffusion_loss,reconstruction_loss])
		return cost

	def visualize_points(self, labels):
		# controls the x and y axes of the plot
		# linspace(min on axis, max on axis, spacing on plot -- large number = more field arrows)
		minx = min(self.embedded_points[:,0].detach().cpu().numpy())-1
		maxx = max(self.embedded_points[:,0].detach().cpu().numpy())+1
		miny = min(self.embedded_points[:,1].detach().cpu().numpy())-1
		maxy = max(self.embedded_points[:,1].detach().cpu().numpy())+1
		x, y = np.meshgrid(np.linspace(minx,maxx,20),np.linspace(miny,maxy,20))
		x = torch.tensor(x,dtype=float).cpu()
		y = torch.tensor(y,dtype=float).cpu()
		xy_t = torch.concat([x[:,:,None],y[:,:,None]],dim=2).float().to('cuda') # TODO: cuda/cpu issue
		uv = self.FlowArtist(xy_t).detach()
		u = uv[:,:,0].cpu()
		v = uv[:,:,1].cpu()
		
		# quiver 
		# 	plots a 2D field of arrows
		# 	quiver([X, Y], U, V, [C], **kw); 
		# 	X, Y define the arrow locations, U, V define the arrow directions, and C optionally sets the color.
		
		sc = plt.scatter(self.embedded_points[:,0].detach().cpu(),self.embedded_points[:,1].detach().cpu(), c=labels)
		plt.quiver(x,y,u,v)
		plt.legend()
		# Display all open figures.
		plt.show()


	def fit(self,n_steps = 1000):
		# train Flow Embedder on the provided graph
		self.train()
		for step in trange(n_steps):
			self.optim.zero_grad()
			# compute loss
			loss = self.loss()
			# print("loss is ",loss)
			# compute gradient and step backwards
			loss.backward()
			self.optim.step()
			"""
			if step % 100 == 0:
				print(f"EPOCH {step}. Loss {loss}. Flow strength {self.flow_strength}. Heatmap of P embedding is ")
				plt.imshow(self.P_embedding.detach().cpu().numpy())
				plt.show()
			"""
			# TODO: Criteria to automatically end training
		print("Exiting training with loss ",loss)
		return self.embedded_points

## Test

In [ ]:
if torch.__version__[:4] == "1.13":
	device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.has_mps else 'cpu')
else:
	device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import numpy as np
from random import *

def tree():
    labels = []
    data = np.array([[None,None] for i in range(900)])
    flow = np.array([[None,None] for i in range(900)])
    
    for i in range(100):
        data[i][0] = np.random.uniform(0.0, 1.0)
        data[i][1] = np.random.uniform(1.0, 2.0)
        f = [np.random.uniform(2.0, 3.0) - data[i][0], np.random.uniform(-1.0, 1.0) - data[i][1]]
        flow[i] = f/np.linalg.norm(f)
        labels.append(0)
    for i in range(100,200):
        data[i][0] = np.random.uniform(0.0, 1.0)
        data[i][1] = np.random.uniform(-2.0, -1.0)
        f = [np.random.uniform(2.0, 3.0) - data[i][0], np.random.uniform(-1.0, 1.0) - data[i][1]]
        flow[i] = f/np.linalg.norm(f)
        labels.append(1)
    for i in range(200,300):
        data[i][0] = np.random.uniform(2.0, 3.0)
        data[i][1] = np.random.uniform(-1.0, 1.0)
        if(i < 250):
            f = [np.random.uniform(4.0, 5.0) - data[i][0], np.random.uniform(2.0, 3.0) - data[i][1]]
            flow[i] = f/np.linalg.norm(f)
        else:
            f = [np.random.uniform(4.0, 5.0) - data[i][0], np.random.uniform(-2.0, -1.0) - data[i][1]]
            flow[i] = f/np.linalg.norm(f)
        labels.append(2)
    for i in range(300,400):
        data[i][0] = np.random.uniform(4.0, 5.0)
        data[i][1] = np.random.uniform(2.0, 3.0)
        if(i < 250):
            f = [np.random.uniform(6.0, 7.0) - data[i][0], np.random.uniform(3.0, 4.0) - data[i][1]]
            flow[i] = f/np.linalg.norm(f)
        else:
            f = [np.random.uniform(6.0, 7.0) - data[i][0], np.random.uniform(1.0, 2.0) - data[i][1]]
            flow[i] = f/np.linalg.norm(f)
        labels.append(3)
    for i in range(400,500):
        data[i][0] = np.random.uniform(4.0, 5.0)
        data[i][1] = np.random.uniform(-2.0, -1.0)
        if(i < 250):
            f = [np.random.uniform(6.0, 7.0) - data[i][0], np.random.uniform(-2.0, -1.0) - data[i][1]]
            flow[i] = f/np.linalg.norm(f)
        else:
            f = [np.random.uniform(6.0, 7.0) - data[i][0], np.random.uniform(-4.0, -3.0) - data[i][1]]
            flow[i] = f/np.linalg.norm(f)
        labels.append(4)
    for i in range(500,600):
        data[i][0] = np.random.uniform(6.0, 7.0)
        data[i][1] = np.random.uniform(3.0, 4.0)
        flow[i] = [1,0]
        labels.append(5)
    for i in range(600,700):
        data[i][0] = np.random.uniform(6.0, 7.0)
        data[i][1] = np.random.uniform(1.0, 2.0)
        flow[i] = [1,0]
        labels.append(6)
    for i in range(700,800):
        data[i][0] = np.random.uniform(6.0, 7.0)
        data[i][1] = np.random.uniform(-2.0, -1.0)
        flow[i] = [1,0]
        labels.append(7)
    for i in range(800,900):
        data[i][0] = np.random.uniform(6.0, 7.0)
        data[i][1] = np.random.uniform(-4.0, -3.0)
        flow[i] = [1,0]
        labels.append(8)
        
    return np.array(data, dtype=float), np.array(flow, dtype=float), labels

In [ ]:
data, flow, labels = tree()

In [ ]:
X = torch.tensor(data)
flow = torch.tensor(flow)
X = X.float().to(device)
flow = flow.float().to(device)

In [ ]:
dfe = DiffusionFlowEmbedder(X,flow,t=1,sigma_graph=15,sigma_embedding=15)
dfe = dfe.to(device)
embeddings = dfe.fit(n_steps=2000)

RuntimeError: The size of tensor a (2) must match the size of tensor b (900) at non-singleton dimension 1

In [ ]:
dfe.visualize_points(labels)

### On circle and Swiss Roll

In [ ]:
from directed_graphs.datasets import directed_circle, directed_cylinder, directed_spiral, directed_swiss_roll
from directed_graphs.datasets import plot_directed_2d, plot_directed_3d

In [ ]:
X, flow, labels = directed_circle(num_nodes=2000, radius=1)
plot_directed_2d(X, flow, labels)

In [ ]:
X = torch.tensor(X)
flow = torch.tensor(flow)
X = X.float().to(device)
flow = flow.float().to(device)

In [ ]:
dfe = DiffusionFlowEmbedder(X,flow,t=1,sigma_graph=15,sigma_embedding=15,device=device)
dfe = dfe.to(device)
embeddings = dfe.fit(n_steps=500)

In [ ]:
dfe.visualize_points(labels)